In [1]:
# 필요 라이브러리 import 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
train = pd.read_csv("./data/8-1 train(std).csv",index_col=0)
test1 = pd.read_csv("./data/8-1 test1(std).csv",index_col=0)
test2 = pd.read_csv("./data/8-1 test2(std).csv",index_col=0)


In [3]:
x_train = train.drop("label",axis=1)
y_train = train["label"]
x_test =test2.drop("label",axis=1)
y_test = test2["label"]

In [4]:
# 성능확인 코드
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score


import warnings

def get_clf_eval(MDA_y_test, pred):
    confusion = confusion_matrix(MDA_y_test, pred)
    accuracy = accuracy_score(MDA_y_test, pred)
    precision = precision_score(MDA_y_test, pred)
    recall = recall_score(MDA_y_test, pred)
    roc_score = roc_auc_score(MDA_y_test, pred)
    pr_score = average_precision_score(MDA_y_test, pred)
    f1 = f1_score(MDA_y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도 : {1:.4f}, 재현율:{2:.4f},F1 스코어:{3:.4f}'.format(accuracy, precision, recall, f1, roc_score))
    print('ROC 스코어: {0:.4f}, PR score : {1:.4f}'.format(roc_score, pr_score))

In [5]:
# 임계값에 따른 오차행렬및 스코어 -------->#임계값 최적 : 재현율기준(0.1)/f1기준(0.3)
from sklearn.preprocessing import Binarizer
thresholds = [0.1,0.2,0.3,0.4,0.5]

def get_eval_by_threshold(MDA_y_test, pred_proba_c1, thresholds):
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print("임곗값:", custom_threshold)
        get_clf_eval(MDA_y_test, custom_predict)
        print("---------------------------------------------------------")



In [6]:
# Roc Auc 그래프 값 확인 코드
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
def roc_curve_plot(MDA_y_test, pred_proba_c1):
    fprs, tprs, thresholds = roc_curve(MDA_y_test, pred_proba_c1)
    
    plt.plot(fprs, tprs, label='ROC')
    plt.plot([0,1],[0,1], 'k--', label = 'Random')
    
    start, end= plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR(1-Sensitivity)'); plt.ylabel('TPR(Recall)')
    plt.legend()
    plt.show()

In [7]:
# 1-1 LDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda_clf = LinearDiscriminantAnalysis()

lda_clf.fit(x_train, y_train)
lda_pred = lda_clf.predict(x_test)
lda_pred_proba = lda_clf.predict_proba(x_test)
print("---------------------------------------------------------") 
get_clf_eval(y_test, lda_pred)
get_eval_by_threshold(y_test , lda_pred_proba[:,1].reshape(-1,1), thresholds)
    

---------------------------------------------------------
오차행렬
[[2124   57]
 [  15    7]]
정확도: 0.9673, 정밀도 : 0.1094, 재현율:0.3182,F1 스코어:0.1628
ROC 스코어: 0.6460, PR score : 0.0416
임곗값: 0.1
오차행렬
[[2072  109]
 [  12   10]]
정확도: 0.9451, 정밀도 : 0.0840, 재현율:0.4545,F1 스코어:0.1418
ROC 스코어: 0.7023, PR score : 0.0436
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2091   90]
 [  12   10]]
정확도: 0.9537, 정밀도 : 0.1000, 재현율:0.4545,F1 스코어:0.1639
ROC 스코어: 0.7066, PR score : 0.0509
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2108   73]
 [  12   10]]
정확도: 0.9614, 정밀도 : 0.1205, 재현율:0.4545,F1 스코어:0.1905
ROC 스코어: 0.7105, PR score : 0.0602
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2118   63]
 [  14    8]]
정확도: 0.9650, 정밀도 : 0.1127, 재현율:0.3636,F1 스코어:0.1720
ROC 스코어: 0.6674, PR score : 0.0473
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2124   57]
 [  15    7]]
정확도: 0.9673, 정밀도 : 0.1094, 재현율:0.3182,

In [8]:
# 1-2 디시전트리 
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)

dt_clf.fit(x_train, y_train)
dt_pred = dt_clf.predict(x_test)
dt_pred_proba = dt_clf.predict_proba(x_test)   

get_clf_eval(y_test, dt_pred)
get_eval_by_threshold(y_test , dt_pred_proba[:,1].reshape(-1,1), thresholds)
    


오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
임곗값: 0.1
오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2140   41]
 [  19    3]]
정확도: 0.9728, 정밀도 : 0.0682, 재현율:0.1364,F1 스코어:0.0909
ROC 스코어: 0.5588, PR score : 0.0179
---------

In [9]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier(random_state=52)
from sklearn.model_selection import GridSearchCV
tree = DecisionTreeClassifier(random_state=52)

# key: 하이퍼파라미터 명; value: [후보값들] 을 리스트로 전달
param_grid = {
    'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10]}
grid_search = GridSearchCV(tree
                        , param_grid = param_grid
                        , cv = 5 #교차검증시 fold 개수; n번의 교차검증; 데이터들이 돌아가면서 하나씩 검증(validation)에 사용되어 일반화하는 것; 데이터가 많지 않을 때 보통 쓰인다고 함
                        , n_jobs = -1)  #-1: 모든 코어 사용

grid_search.fit(x_train, y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

dt_clf_best = DecisionTreeClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3], min_samples_split=values[4])
dt_clf_best.fit(x_train, y_train)
dt_best_pred = dt_clf_best.predict(x_test)
dt_best_pred_proba = dt_clf_best.predict_proba(x_test)   

print("디시전트리 파라미터적용") 
get_clf_eval(y_test, dt_best_pred)
get_eval_by_threshold(y_test ,dt_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'min_samples_split': 4}, 최적 평균 정확도 :0.985
디시전트리 파라미터적용
오차행렬
[[2168   13]
 [  21    1]]
정확도: 0.9846, 정밀도 : 0.0714, 재현율:0.0455,F1 스코어:0.0556
ROC 스코어: 0.5197, PR score : 0.0128
임곗값: 0.1
오차행렬
[[2152   29]
 [  19    3]]
정확도: 0.9782, 정밀도 : 0.0938, 재현율:0.1364,F1 스코어:0.1111
ROC 스코어: 0.5615, PR score : 0.0214
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2152   29]
 [  19    3]]
정확도: 0.9782, 정밀도 : 0.0938, 재현율:0.1364,F1 스코어:0.1111
ROC 스코어: 0.5615, PR score : 0.0214
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2168   13]
 [  21    1]]
정확도: 0.9846, 정밀도 : 0.0714, 재현율:0.0455,F1 스코어:0.0556
ROC 스코어: 0.5197, PR score : 0.0128
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2168   13]
 [  21    1]]
정확도: 0.9846, 정밀도 : 0.0714, 재현율:0.0455,F1 스코어:0.0556
ROC 스코어: 0.5197, PR score : 0.0128
------------------------------------------

In [10]:
print("디시전트리") 
print(get_clf_eval(y_test, lda_pred))

디시전트리
오차행렬
[[2124   57]
 [  15    7]]
정확도: 0.9673, 정밀도 : 0.1094, 재현율:0.3182,F1 스코어:0.1628
ROC 스코어: 0.6460, PR score : 0.0416
None


In [11]:
# 1-3 랜덤포레스트 구현 
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state=52)    

rf_clf.fit(x_train, y_train)
rf_pred = rf_clf.predict(x_test)
rf_pred_proba = rf_clf.predict_proba(x_test)
print("랜덤포레스트") 
get_clf_eval(y_test, rf_pred)
get_eval_by_threshold(y_test , rf_pred_proba[:,1].reshape(-1,1), thresholds)



랜덤포레스트
오차행렬
[[2173    8]
 [  20    2]]
정확도: 0.9873, 정밀도 : 0.2000, 재현율:0.0909,F1 스코어:0.1250
ROC 스코어: 0.5436, PR score : 0.0273
임곗값: 0.1
오차행렬
[[2094   87]
 [  10   12]]
정확도: 0.9560, 정밀도 : 0.1212, 재현율:0.5455,F1 스코어:0.1983
ROC 스코어: 0.7528, PR score : 0.0707
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2129   52]
 [  14    8]]
정확도: 0.9700, 정밀도 : 0.1333, 재현율:0.3636,F1 스코어:0.1951
ROC 스코어: 0.6699, PR score : 0.0548
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2146   35]
 [  16    6]]
정확도: 0.9768, 정밀도 : 0.1463, 재현율:0.2727,F1 스코어:0.1905
ROC 스코어: 0.6283, PR score : 0.0472
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2162   19]
 [  19    3]]
정확도: 0.9828, 정밀도 : 0.1364, 재현율:0.1364,F1 스코어:0.1364
ROC 스코어: 0.5638, PR score : 0.0272
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2173    8]
 [  20    2]]
정확도: 0.9873, 정밀도 : 0.2000, 재현율:0.0909,F1 스코어:0.1250
ROC 스코어: 0.5436, PR score : 0.0273
--

In [12]:
# 하이퍼파라미터 Grid-Search
from sklearn.model_selection import GridSearchCV

    # 탐색 범위 정의

grid_search = {'criterion': ['entropy', 'gini'],
            'max_depth': [2,4,6,8],
            'max_features': ['auto', 'sqrt'],
            'min_samples_leaf': [4,6,8,10],
            'min_samples_split': [4,6,8,10],
            'n_estimators': [20,40,60,80,100]}

model = RandomForestClassifier(random_state=52)
grid_search = GridSearchCV(estimator = model, param_grid = grid_search, 
                            cv = 5,verbose=0, n_jobs = -1)
grid_search.fit(x_train,y_train)

# 최선의 결과 출력
print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))
a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

rf_clf_best = RandomForestClassifier(random_state=52,criterion = values[0], max_depth = values[1],max_features=values[2], min_samples_leaf=values[3],  min_samples_split=values[4],n_estimators=values[5])
rf_clf_best.fit(x_train, y_train)
rf_best_pred = rf_clf_best.predict(x_test)
rf_best_pred_proba = rf_clf_best.predict_proba(x_test)   

print("랜덤포레스트 파라미터적용") 
get_clf_eval(y_test, rf_best_pred)
get_eval_by_threshold(y_test ,rf_best_pred_proba[:,1].reshape(-1,1), thresholds)

최적 하이퍼 파라미터 :{'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 40}, 최적 평균 정확도 :0.986
랜덤포레스트 파라미터적용
오차행렬
[[2175    6]
 [  20    2]]
정확도: 0.9882, 정밀도 : 0.2500, 재현율:0.0909,F1 스코어:0.1333
ROC 스코어: 0.5441, PR score : 0.0318
임곗값: 0.1
오차행렬
[[2099   82]
 [  13    9]]
정확도: 0.9569, 정밀도 : 0.0989, 재현율:0.4091,F1 스코어:0.1593
ROC 스코어: 0.6857, PR score : 0.0464
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2136   45]
 [  15    7]]
정확도: 0.9728, 정밀도 : 0.1346, 재현율:0.3182,F1 스코어:0.1892
ROC 스코어: 0.6488, PR score : 0.0496
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2159   22]
 [  16    6]]
정확도: 0.9828, 정밀도 : 0.2143, 재현율:0.2727,F1 스코어:0.2400
ROC 스코어: 0.6313, PR score : 0.0657
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2167   14]
 [  20    2]]
정확도: 0.9846, 정밀도 : 0.1250, 재현율:0.0909,F1 스코어:0.1053
ROC 스코어: 0.5422, PR score : 0.0204
--------------------

In [13]:
# 1-4 로지스틱회귀
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

lr_clf.fit(x_train, y_train)
lr_pred = lr_clf.predict(x_test)
lr_pred_proba = lr_clf.predict_proba(x_test)
print("로지스틱회귀") 
get_clf_eval(y_test, lr_pred)
get_eval_by_threshold(y_test , lr_pred_proba[:,1].reshape(-1,1), thresholds)
    

로지스틱회귀
오차행렬
[[2170   11]
 [  18    4]]
정확도: 0.9868, 정밀도 : 0.2667, 재현율:0.1818,F1 스코어:0.2162
ROC 스코어: 0.5884, PR score : 0.0567
임곗값: 0.1
오차행렬
[[2110   71]
 [  11   11]]
정확도: 0.9628, 정밀도 : 0.1341, 재현율:0.5000,F1 스코어:0.2115
ROC 스코어: 0.7337, PR score : 0.0721
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2150   31]
 [  15    7]]
정확도: 0.9791, 정밀도 : 0.1842, 재현율:0.3182,F1 스코어:0.2333
ROC 스코어: 0.6520, PR score : 0.0654
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2161   20]
 [  17    5]]
정확도: 0.9832, 정밀도 : 0.2000, 재현율:0.2273,F1 스코어:0.2128
ROC 스코어: 0.6091, PR score : 0.0532
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2167   14]
 [  17    5]]
정확도: 0.9859, 정밀도 : 0.2632, 재현율:0.2273,F1 스코어:0.2439
ROC 스코어: 0.6104, PR score : 0.0675
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2170   11]
 [  18    4]]
정확도: 0.9868, 정밀도 : 0.2667, 재현율:0.1818,F1 스코어:0.2162
ROC 스코어: 0.5884, PR score : 0.0567
--

In [14]:
#로지스틱 회귀 최적 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV


params = {'penalty': ['l2','l1','elasticnet','None'],
        'C':[0.01,0.1,1,3,5,10]}

grid_search = GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_search.fit(x_train,y_train)
print('최적 하이퍼 파라미어:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_search.best_params_, grid_search.best_score_))

a = grid_search.best_params_
keys=list(a.keys())
values = list(a.values())

lr_clf_best = LogisticRegression(penalty= values[1], C=values[0])
lr_clf_best.fit(x_train, y_train)
lr_best_pred = lr_clf_best.predict(x_test)
lr_best_pred_proba = lr_clf_best.predict_proba(x_test)   

print("로지스틱회귀 파라미터적용") 
get_clf_eval(y_test, lr_best_pred)
get_eval_by_threshold(y_test ,lr_best_pred_proba[:,1].reshape(-1,1), thresholds)


최적 하이퍼 파라미어:{'C': 0.1, 'penalty': 'l2'}, 최적 평균 정확도:0.982
로지스틱회귀 파라미터적용
오차행렬
[[2171   10]
 [  20    2]]
정확도: 0.9864, 정밀도 : 0.1667, 재현율:0.0909,F1 스코어:0.1176
ROC 스코어: 0.5432, PR score : 0.0242
임곗값: 0.1
오차행렬
[[2111   70]
 [  13    9]]
정확도: 0.9623, 정밀도 : 0.1139, 재현율:0.4091,F1 스코어:0.1782
ROC 스코어: 0.6885, PR score : 0.0525
---------------------------------------------------------
임곗값: 0.2
오차행렬
[[2150   31]
 [  16    6]]
정확도: 0.9787, 정밀도 : 0.1622, 재현율:0.2727,F1 스코어:0.2034
ROC 스코어: 0.6293, PR score : 0.0515
---------------------------------------------------------
임곗값: 0.3
오차행렬
[[2164   17]
 [  17    5]]
정확도: 0.9846, 정밀도 : 0.2273, 재현율:0.2273,F1 스코어:0.2273
ROC 스코어: 0.6097, PR score : 0.0594
---------------------------------------------------------
임곗값: 0.4
오차행렬
[[2169   12]
 [  19    3]]
정확도: 0.9859, 정밀도 : 0.2000, 재현율:0.1364,F1 스코어:0.1622
ROC 스코어: 0.5654, PR score : 0.0359
---------------------------------------------------------
임곗값: 0.5
오차행렬
[[2171   10]
 [  20    2]]
정확도: 0.9864, 정밀도 : 0.1667